In [1]:
import cosima_cookbook as cc
import numpy as np

## Importing the model outputs the chosen time period and domain

In [2]:
session = cc.database.create_session()
expt01 = '01deg_jra55v140_iaf_cycle2'  # 01-deg and daily experiment 
yt_sel = slice(-60, -45)
xt_sel = slice(144.5-360, 155.5-360)
start_time, end_time = '1993-01-01', '2018-12-31'

In [3]:
var = 'u'
u = cc.querying.getvar(expt01, var, session, frequency='1 monthly', start_time=start_time, end_time=end_time)
u = u.sel(time=slice(start_time, end_time)).sel(yu_ocean=yt_sel, xu_ocean=xt_sel, st_ocean = slice(0,500))

var = 'v'
v = cc.querying.getvar(expt01, var, session, frequency='1 monthly', start_time=start_time, end_time=end_time)
v = v.sel(time=slice(start_time, end_time)).sel(yu_ocean=yt_sel, xu_ocean=xt_sel, st_ocean = slice(0,500))

In [4]:
variable = 'temp'
temp = cc.querying.getvar(expt01, variable, session, frequency = '1 monthly', start_time=start_time, end_time=end_time)
temp = temp.sel(yt_ocean=yt_sel, xt_ocean=xt_sel, st_ocean = slice(0,500),
                time=slice(start_time, end_time)) - 273.15 #the temperature of the model is in Kelvin

## We perform an 18month window average to extract the signal of the meander

In [ ]:
wdw       = 18
temp_wdw  = temp.rolling(time = wdw, center = True).mean('time').compute()
u_wdw     = u.rolling(time = wdw, center = True).mean('time').compute()
v_wdw     = v.rolling(time = wdw, center = True).mean('time').compute()

### Let's evaluate the first period of the smoothed time series (i.e., wdw/2)

In [ ]:
temp_snap = temp_wdw[int(wdw/2)]
u_snap = u_wdw[int(wdw/2)]
v_snap = v_wdw[int(wdw/2)]

In [ ]:
## --> That is the variable that you use to find the coordinates of the front
temp_PF   = temp_snap.interp(st_ocean = 282).compute()

## TBC

--> Things to note: the isotherm that represents the PF is the 2.9$^\circ$C at 282 m
--> The model outputs are in T-cell coordinates (i.e., to convert the longitudes to the cartesin grid, you need to sum 360$^\circ$) ----> Ln, Lt = temp.xt_ocean + 360, temp.yt_ocean